In [36]:
import os
import sys
import psutil
import traci
import time

# Ensure SUMO_HOME is set
if "SUMO_HOME" in os.environ:
    sys.path.append(os.path.join(os.environ["SUMO_HOME"], "tools"))

# Function to kill old SUMO processes before starting
def kill_sumo():
    for process in psutil.process_iter(attrs=["pid", "name"]):
        if "sumo" in process.info["name"].lower():
            print(f"Killing SUMO process {process.info['name']} (PID: {process.info['pid']})")
            os.kill(process.info["pid"], 9)

kill_sumo()

# Run simulation
sumoBinary = "sumo/sumo/bin/sumo-gui"
sumoCmd = [sumoBinary, "-c", "osm.sumocfg"]

try:
    # Increase retries to prevent connection issues
    traci.start(sumoCmd, numRetries=20)

    charging_stations = traci.chargingstation.getIDList()
    print(f"⚡ Available Charging Stations: {charging_stations}")

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()

        for veh_id in traci.vehicle.getIDList():
            try:    
                # Battery parameters
                battery_level = float(traci.vehicle.getParameter(veh_id, "device.battery.actualBatteryCapacity"))
                max_battery = float(traci.vehicle.getParameter(veh_id, "device.battery.maximumBatteryCapacity"))

                # Check if stationfinder is enabled
                has_stationfinder = traci.vehicle.getParameter(veh_id, "has.stationfinder.device")

            except traci.TraCIException as e:
                print(f"Error retrieving battery level for {veh_id}: {e}")
                
except traci.exceptions.FatalTraCIError as e:
    print(f"TraCI connection lost: {e}")

except BrokenPipeError:
    print("BrokenPipeError: SUMO crashed or closed unexpectedly.")

finally:
    time.sleep(1)
    try:
        if traci.connection._connections:
            print("Closing TraCI connection properly...")
            traci.close()
    except Exception as e:
        print(f"TraCI was already closed: {e}")


 Retrying in 1 seconds
⚡ Available Charging Stations: ('cS_-364911324', 'cS_4998703#0', 'cS_4998713#0')
Closing TraCI connection properly...
